In [5]:
import json
import requests
import pandas as pd
from pathlib import Path
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np
from pathlib import Path
from sklearn import cluster, covariance, manifold
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import warnings
warnings.filterwarnings('ignore')


In [6]:
events=pd.read_csv(
    Path('Institutional_Adoption_Events_in_2021.csv'),
    index_col='DATE',
    parse_dates=True,
    infer_datetime_format=True
)
events=events.fillna(0)
events.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Institutional_Adoption_Events_in_2021.csv'

In [ ]:
events=events.drop(['NEWS', 'CATEGORY'], axis=1)
events.head()

In [ ]:
prices=pd.read_csv(
    Path('crypto_market_caps.csv'),
    index_col='date',
    parse_dates=True,
    infer_datetime_format=True

)
prices.head()

In [ ]:
combined=pd.merge(prices,events,left_index=True, right_index=True)
combined.head()

In [ ]:
combined = combined.dropna(axis=0, how='all')
combined = combined.dropna(axis=1)
combined

In [ ]:
combined_new=combined.drop(['Institutional Adopt'], axis=1)
combined_new.head()

In [ ]:
variation = combined_new
variation.shape

In [ ]:
column_names= list(combined_new)

In [ ]:
# calculate mean and variance for each token
annual_returns = []
variance = []
for x in column_names:
    annual_returns.append(combined_new[x].mean()*252)
    variance.append(combined_new[x].std()*np.sqrt(252))
    
    

In [ ]:
stats=pd.DataFrame([annual_returns,variance])

In [ ]:
stats_df=stats.T

In [ ]:
stats_df.columns=('annual_return','variance')

In [ ]:
stats_df.index=column_names

In [ ]:
stats_df.head()

In [ ]:
# Create a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

In [ ]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the service_ratings DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(stats_df)
    inertia.append(k_model.inertia_)

In [ ]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

In [ ]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

In [ ]:
# Create and initialize the K-means model instance for 4 clusters
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the data to the instance of the model
model.fit(stats_df)

# Make predictions about the data clusters using the trained model
crypto_clusters = model.predict(stats_df)
crypto_clusters

In [ ]:
len(crypto_clusters)

In [ ]:
crypto_clusters_df = pd.DataFrame(crypto_clusters)
crypto_clusters_df.head()

In [ ]:
stats_df['crypto_cluster']=crypto_clusters
stats_df['names']=column_names
stats_df


In [ ]:
# Plot the crypto clusters Data Frame
stats_df.hvplot.scatter(x="annual_return", y="variance", by="crypto_cluster",text='names',text_baseline='top',hover=True)
#stats_df.hvplot.labels(x='annual_return', y='variance',by='crypto_cluster', text='names',text_baseline='top', hover=True)

In [ ]:
correlations_df=combined_new.corr()
btc_correlations=correlations_df.loc['bitcoin']

In [ ]:
btc_correlations=btc_correlations.sort_values()

btc_correlations.hvplot.scatter()

In [ ]:
plt.figure(figsize = (16,10))
ax = sns.heatmap(correlations_df, annot=True, linewidths=.5)


In [ ]:
correlations_df

In [ ]:
# Plot the DataFrame
# Visualization
plt.figure(1, facecolor='w', figsize=(12, 9))
plt.clf()
ax = plt.axes([0., 0., 1., 1.])
plt.axis('off')


#Display a graph of the partial correlations
partial_correlations = edge_model.precision_.copy()
d = 1 / np.sqrt(np.diag(partial_correlations))
partial_correlations *= d
partial_correlations *= d[:, np.newaxis]
non_zero = (np.abs(np.triu(partial_correlations, k=1)) > 0.02)


# Plot the nodes using the coordinates of our embedding
plt.scatter(embedding[0], embedding[1], s=100 * d ** 2, c=labels,
            cmap=plt.cm.nipy_spectral)

# Plot the edges
start_idx, end_idx = np.where(non_zero)

# a sequence of (*line0*, *line1*, *line2*), where::
#            linen = (x0, y0), (x1, y1), ... (xm, ym)
segments = [[embedding[:, start], embedding[:, stop]]
            for start, stop in zip(start_idx, end_idx)]
values = np.abs(partial_correlations[non_zero])
lc = LineCollection(segments,
                    zorder=0, cmap=plt.cm.hot_r,
                    norm=plt.Normalize(0, .7 * values.max()))
lc.set_array(values)
lc.set_linewidths(15 * values)
ax.add_collection(lc)


# Add a label to each node. The challenge here is that we want to
# position the labels to avoid overlap with other labels
for index, (name, label, (x, y)) in enumerate(
        zip(names, labels, embedding.T)):
    dx = x - embedding[0]
    dx[index] = 1
    dy = y - embedding[1]
    dy[index] = 1
    this_dx = dx[np.argmin(np.abs(dy))]
    this_dy = dy[np.argmin(np.abs(dx))]
    if this_dx > 0:
        horizontalalignment = 'left'
        x = x + .002
    else:
        horizontalalignment = 'right'
        x = x - .002
    if this_dy > 0:
        verticalalignment = 'bottom'
        y = y + .002
    else:
        verticalalignment = 'top'
        y = y - .002
    plt.text(x, y, name, size=10,
             horizontalalignment=horizontalalignment,
             verticalalignment=verticalalignment,
             bbox=dict(facecolor='w',
                       edgecolor=plt.cm.nipy_spectral(label / float(n_labels)),
                       alpha=.6))
plt.xlim(embedding[0].min() - .15 * embedding[0].ptp(),
         embedding[0].max() + .10 * embedding[0].ptp(),)
plt.ylim(embedding[1].min() - .03 * embedding[1].ptp(),
         embedding[1].max() + .03 * embedding[1].ptp())
plt.show()